In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))

from database_operations import DatabaseOperations
from overtake_model import OvertakingModel
from race_data import RaceDataSetup
from race_dataframe import RaceDataframe
from race_sim import RaceSimulator
from evaluation import RaceSimEvaluation, EvaluateMany

import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None) 

In [ ]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd

class RaceDataSetup:
	def __init__(self, db_operations_obj, race_df_obj):
		self.__db_operations_obj = db_operations_obj      # this is an object of the DatabaseOperations class
		self.__race_dataframe_obj = race_df_obj      # this is an object of the RaceDataframe class

		self.race_df = self.__race_dataframe_obj.race_df


	# ----------------------- Mainly for calculating tyre deg -----------------------
	#									   START
	@staticmethod
	def correct_fuel_effect(race_df, max_lap=None, max_fuel_kg=110, fuel_effect_per_kg=0.03):
		""" Assigns new columns for fuel effects at the sector level.

		Args:
			race_df (pd.DataFrame): dataframe of the race
			max_lap (int, optional): the max number of laps in the race. Defaults to None.
			max_fuel_kg (int, optional): how much fuel the cars are starting with in KG. Defaults to 110.
			fuel_effect_per_kg (float, optional): the time lost per lap, for each kg of fuel. Defaults to 0.03.

		Returns:
			pd.DataFrame: df with new fuel corrected columns
		"""

		if max_lap is None:
			max_lap = race_df["lap_num"].max()

		fuel_reduction_per_lap = max_fuel_kg / max_lap
		fuel_reduction_per_sector = fuel_reduction_per_lap / 3
		fuel_time_per_sector = fuel_effect_per_kg / 3

		def _correct_fuel_for_driver(driver_df):
			# Calculate fuel weight for each sector
			driver_df.loc[:, "fuel_weight_sector"] = max_fuel_kg - (
				(driver_df["lap_num"] - 1) * fuel_reduction_per_lap +
				(driver_df["sector"] - 1) * fuel_reduction_per_sector
			)

			# Calculate fuel correction for each sector
			driver_df.loc[:, "fuel_correction_sector"] = (
				driver_df["fuel_weight_sector"] * fuel_time_per_sector
			)

			# Apply fuel correction to sector times
			driver_df.loc[:, "fuel_corrected_sector_time"] = (
				driver_df["sector_time"] - driver_df["fuel_correction_sector"]
			)

			return driver_df

		race_df = race_df.groupby("driver_number", group_keys=False).apply(_correct_fuel_for_driver).reset_index(drop=True)

		return race_df

	@staticmethod
	def assign_stint_numbers(race_df):
		"""Assigns stint numbers to laps based on pit stops for each driver

		Args:
			race_df (pd.DataFrame): dataframe of the race

		Returns:
			pd.DataFrame: df with new stint column
		"""

		# Assign stint numbers to laps based on pit stops for each driver
		race_df["stint"] = np.nan
		for driver in race_df["driver_number"].unique():
			driver_data = race_df[race_df["driver_number"] == driver]
			stint_number = 1
			for i in driver_data.index:
				if driver_data.loc[i, "pit"] and i != driver_data.index[0]:
					stint_number += 1
				race_df.loc[i, "stint"] = stint_number
		race_df["stint"] = race_df["stint"].astype(int)
		return race_df

	@staticmethod
	def remove_laps_outside_percent(race_df, percentage=5):
		""" Removes laps where fuel corrected sector times is above the given percentage

		Args:
			race_df (pd.DataFrame): dataframe of the race
			percentage (float, optional): the threshold percentage above the fastest laptime

		Returns:
			pd.DataFrame: df with outliers removed
		"""
			
		def _filter_driver_sector_laps(driver_sector_df):
			# Calculate the threshold based on the fastest lap time for the driver and sector
			fastest_lap_time = driver_sector_df["fuel_corrected_sector_time"].min()
			threshold = fastest_lap_time * (1 + percentage / 100)
			
			# Remove laps not within the specified percentage of the fastest lap time
			filtered_df = driver_sector_df[driver_sector_df["fuel_corrected_sector_time"] <= threshold]
			return filtered_df

		# Group by driver and sector, then apply the filtering logic
		race_df = (
			race_df.groupby(["driver_number", "sector"], group_keys=False)
			.apply(_filter_driver_sector_laps)
			.reset_index(drop=True)
		)


		return race_df

	@staticmethod
	def normalise_lap_times_by_sector(race_df):
		""" Normalises by taking the drivers fastest time in that sector in quali from the sector time
		This means we can get deg curves that arent dependant on the laptime, they are how much time deg adds on

		Args:
			race_df (pd.DataFrame): race df

		Returns:
			pd.DataFrame: df with the normalised time
		"""

		# Normalise lap times by subtracting the fastest sector time
		race_df['normalised_sector_time'] = (
			race_df['fuel_corrected_sector_time'] - race_df['base_sector_time']
		)

		return race_df
	#									   END
	# ----------------------- Mainly for calculating tyre deg -----------------------

	def get_driver_tyre_coefficients(self):
		""" Calculates tyre degradation coefficients for each driver, tyre type, and sector.

		The race df is taken and pre processed, then tyre deg is done on the normalised sector times
		If a driver hasnt used a tyre type, an average of every other driver is added

		Returns:
			dict: {driverNum: {tyreType: {sector1Deg: [], sector2Deg: [], sector3Deg: []}}}
		"""
		# Pre process
		race_df = self.race_df
		race_df = self.assign_stint_numbers(race_df)
		race_df = self.correct_fuel_effect(race_df)
		# race_df = self.remove_laps_outside_percent(race_df)
		# race_df = self.normalise_lap_times_by_sector(race_df)
		return(race_df)

In [24]:
db1 = DatabaseOperations(2024, "Mexico City")
race1 = RaceDataframe(db1)
race_data1 = RaceDataSetup(db1, race1)

race_df = race_data1.get_driver_tyre_coefficients()


/tmp/ipykernel_15121/681048331.py:60: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  race_df = race_df.groupby("driver_number", group_keys=False).apply(_correct_fuel_for_driver).reset_index(drop=True)


In [25]:
race_df[race_df["driver_name"]=="Max Verstappen"]

,lap_num,lap_time,sector,stint_num,stint_lap,position,driver_name,driver_number,sector_time,tyre_type,tyre_laps,pit,pit_time,track_status,base_sector_time,cumulative_time,pace,gap,tyre_diff,front_laps,stint_laps_diff,drs_available,next_pit,overtaken,stint,fuel_weight_sector,fuel_correction_sector,fuel_corrected_sector_time
0,1,105.059,1,1,1,3.0,Max Verstappen,1,NaN,2,1,False,NaN,124,27.222,NaN,NaN,0.000,0.0,NaN,0.0,False,False,True,1,110.000000,1.100000,NaN
20,1,105.059,2,1,1,3.0,Max Verstappen,1,39.491,2,1,False,NaN,124,29.318,39.491,39.491000,0.000,0.0,NaN,0.0,False,False,False,1,109.483568,1.094836,38.396164
40,1,105.059,3,1,1,3.0,Max Verstappen,1,33.673,2,1,False,NaN,124,19.631,73.164,33.673000,0.000,0.0,NaN,0.0,False,False,False,1,108.967136,1.089671,32.583329
60,2,138.259,1,1,2,1.0,Max Verstappen,1,61.769,2,2,False,NaN,4,27.222,134.933,61.769000,0.000,0.0,NaN,0.0,False,False,False,1,108.450704,1.084507,60.684493
78,2,138.259,2,1,2,1.0,Max Verstappen,1,47.111,2,2,False,NaN,4,29.318,182.044,43.301000,0.000,0.0,NaN,0.0,False,False,False,1,107.934272,1.079343,46.031657
96,2,138.259,3,1,2,1.0,Max Verstappen,1,29.379,2,2,False,NaN,4,19.631,211.423,31.526000,0.000,0.0,NaN,0.0,False,False,False,1,107.417840,1.074178,28.304822
114,3,134.297,1,1,3,1.0,Max Verstappen,1,57.820,2,3,False,NaN,4,27.222,269.243,59.794500,0.000,0.0,NaN,0.0,False,False,False,1,106.901408,1.069014,56.750986
132,3,134.297,2,1,3,1.0,Max Verstappen,1,47.214,2,3,False,NaN,4,29.318,316.457,44.605333,0.000,0.0,NaN,0.0,False,False,False,1,106.384977,1.063850,46.150150
150,3,134.297,3,1,3,1.0,Max Verstappen,1,29.263,2,3,False,NaN,4,19.631,345.720,30.771667,0.000,0.0,NaN,0.0,False,False,False,1,105.868545,1.058685,28.204315
168,4,129.792,1,1,4,1.0,Max Verstappen,1,53.334,2,4,False,NaN,4,27.222,399.054,57.641000,0.000,0.0,NaN,0.0,False,False,False,1,105.352113,1.053521,52.280479
